<a href="https://colab.research.google.com/github/easyhardhoon/machine_learning_basic/blob/master/ML_RE6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ML12~ML13

**1. 배치 정규화**

가중치 초깃값을 적절히 설정하여 활성화 값을 잘 퍼지도록 하여 학습을 원활하게 하였다. 

그렇다면 각 층이 활성화를 적당히 잘 퍼드리도록 강제해보면 어떨까

Affine -> batch norm -> Relu 같은 순서로 삽입

▶(batch norm 기준) 입력 데이터의 평균(m) & 분산(o)을, 평균(0) & 분산(1)인 데이터로 정규화한다.

▶정규화된 데이터에 고유한 확대와 이동 변환을 수행한다. (yi = R * x^i + B )

두 값은 처음에는 (1,0) 이었다가, 학습을 진행하며 적합한 값으로 조정해나간다.

이렇게 데이터 분포를 덜 치우치게 하는 작업을 하는 계층을 **배치 정규화 계층**이라하고,이를 신경망에 삽입하는 기술이다

**효과**

✔ 거의 모든 경우에서 학습 진도의 개선.

✔ 가중치 초깃값에 대한 의존도 감소

**2. 오버피팅**

신경망이 훈련 데이터에만 지나치게 적응되어 그 외의 데이터에는 제대로 적응하지 못하는 상태

**오버피팅이 발생하는 경우**

▶매개변수가 많고 표현력이 높은 모델

▶훈련 데이터가 적은 모델

In [ ]:
import os
import sys
sys.path.append(os.pardir)
import numpy as np
import matplotlib.pyplot as plt
from mnist import load_mnist
#from multi_layer_net import MultiLayerNet
from optimizer import SGD

(x_train,t_train), (x_test,t_test) = load_mnist(normalize=True)

x_train = x_train[:300] 
t_train = t_train[:300]

weight_decay_lambda = 0.1

network = MultiLayerNet(input_size=784, hidden_size_list = [100,100,100,100,100,100], \
                        output_size =10, weight_decay_lambda=weight_decay_lambda)
optimizer= SGD(lr=0.01) 

max_epochs= 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
epoch_cnt = 0

for i in range(1000000000):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)

    if i % iter_per_epoch == 0: 
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("epoch:" + str(epoch_cnt) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc))

        epoch_cnt += 1
        if epoch_cnt >= max_epochs:
            break

위 코드에서

MNIST 데이터셋의 훈련 데이터 6만개중 300개만 사용, 7층 네트워크를 사용해 복잡성을 높임

또한 각 층의 뉴런을 100개로 설정하여 매개변수가 많은 표현력이 높은 모델 사용.

실제 그래프를 보면 훈련데이터에만 적응하여 시험데이터에는 잘 적응하지 못하는 모습,

즉 오버피팅이 발생한다. 

✅각 층의 뉴런 수 = 각 층의 parameter의 개수 = 각 층의 매개변수 개수 = 각 층의 가중치 개수

⏰오버피팅을 억제하는 방법

✅가중치 감소

오버피팅은 가중치 매개변수의 값이 커서 발생하는 경우가 많음

그래서 큰 가중치에 대해서는 그에 상응하는 큰 페널티를 부과하여 오버피팅을 억제한다.

**가중치가 W이라면 1/2람다W^2를 손실 함수에 더하여 손실 함수의 값이 더 커지게 한다.**

자연스레 학습의 관점에서 보면 더 조심스레 학습이 진행된다.

(람다 : 정규화의 세기를 조절하는 **하이퍼파라미터**. 1/2람다W^2 이게 L2 법칙입니다)

⚡ 각 층마다 고유의 가중치가 있다. 

⚡ 전체 층의 최종 출력층을 거치고 나서 그 결과값과 정답 레이블을 비교해 단 한번의 손실함수를 구한다

**정리하면**

가중치 감소는 각 루프마다( ex) 각 에포치...) 손실 함수에 1/2ㅅW^2를 더한다

이렇게 되어서 역전파를 구할때 w에 대한 손실함수의 기울기를 구하며 ㅅW을 더한다.

가중치가 이를 기반으로 업데이트 될때, ㅅW 만큼 추가적으로 더해지기 때문에 학습의 관점에서 보면 조심스레 학습이 되어진다고 볼 수 있다.

이러한 과정을 가중치 감소라고 부른다

✅드롭아웃

가중치 감소는 구현이 간단하지만 신경망 모델이 복잡해지면 가중치 감소만으로는 대응하기 힘들다

드롭아웃은, 뉴런을 임의로 삭제하며 학습하는 방법이다.

훈련때에만 데이터를 학습할때마다 삭제할 뉴런(은닉층)을 무작위로 선택하고,

시험떄에는 모든 뉴런에 신호를 전달하는 방법

In [ ]:
import numpy as np
class Dropout:
  def __init__(self,dropout_ratio = 0.5):
    self.dropout_ratio = dropout_ratio
    self.mask = None
  def forward(self,x,train_flg=True):
    if train_flg:
      self.mask = np.random.rand(*x.shape) > self.dropout_ratio 
      #x.shape의 형식이 튜플이기 때문에 *사용
      return x * self.mask 
    else:
      return x * (1.0 * self.dropout_ratio)
  def backward(self,dout):
    return dout * self.mask #순전파때 흘렀던건 역전파때도 흐르게 한다. RELU 와 동일하다
#여기서 x는 w일 가능성이 높다. x이든 w이든 같은 순전파/역전파의 흐름에 속한다.

⏰훈련데이터의 정확도가 100%에 도달하지 않는 경우의 의미

안좋은 뜻이 아니라, 그만큼 표현력이 높아졌다고 해석 가능. 

즉 원래 95를 그냥 100으로 봤다면 95를 95로 보기 시작했다는 정도의 의미

---

⏰드롭아웃은 단 하나의 네트워크로 앙상블 학습을 한 효과를 낼 수 있다.

앙상블 학습 : 개별적으로 학습시킨 여러 모델(네트워크)의 출력을 평균 내어 추론하는 방식

---

드롭아웃이나 가중치 감소나 하나의 계층으로 구현된다. 

전체 계층에 같은 순전파와 역전파가 흐르는 원리이다

**3.하이퍼 파라미터 최적화**

최적화된 하이퍼 파라미터값을 찾기 위해 데이터셋을 활용할때, 시험 데이터를 사용해서는 안된다.

검증 데이터라고 불리는 전용 데이터를 사용한다.

▶훈련 데이터 : 매개변수 학습용

▶검증 데이터 : 하이퍼파라미터 성능 평가용

▶시험 데이터 : 신경망의 범용 성능 평가용

---

**핵심 : 하이퍼 파라미터의 최적 값이 존재하는 범위를 조금씩 줄여나가기**

**대략적으로 범위를 지정**하고, 그 범위에서 **무작위로 값을 골라내고** 

그 값으로 **정확도를 평가**하는 작업을 여러번 **반복**하여 하이퍼 파라미터의 최적값의 **범위를 좁혀나간다.**

하이퍼 파라미터 최적화에는 오랜시간이 걸리기 떄문에, 규칙적인 탐색보다는 위와 같은 **무작위 탐색이 효과적**이라고 알려져있다.

세련된 규칙적인 탐색 기법으로 베이즈 최적화가 있긴 하다



In [ ]:
import numpy as np
weight_key = 10 ** np.random.uniform(-8,-4) #가중치 감소 계수
lr = 10 ** np.random.uniform(-6,-2) # 학습률

▶np.random.uniform(low,high,size) : 균등분포로부터 low~high범위내 무작위 size번 추출 